### Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from ETGEMs_function_ETG import *
import pandas as pd
from ppmcot import *

In [2]:
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

### Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './HSEC0916-g0.txt'
reaction_g0_file_original1 = './HSEC1017-g0.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './HSEC0916-lnC.txt'
metabolites_lnC_file_original1 = './HSEC1017-lnC.txt'
#Get Model from local file
model_file_original0 = './HSEC0916.xml'
model_file_original1 = './HSEC1017.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './HSEC0916-kcat-MW.csv'
reaction_kcat_MW_file_original1 = './HSEC1017-kcat-MW.csv'

In [4]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

### Strain reconstruction

In [5]:
model0['ub_list']['ACALD']=0
model0['lb_list']['ACALD']=0
model0['ub_list']['ACALD_reverse']=0
model0['lb_list']['ACALD_reverse']=0
model0['ub_list']['ALCD2x']=0
model0['lb_list']['ALCD2x']=0
model0['ub_list']['ALCD2x_reverse']=0
model0['lb_list']['ALCD2x_reverse']=0
model0['ub_list']['LDH_D']=0
model0['lb_list']['LDH_D']=0
model0['ub_list']['FRD2']=0
model0['lb_list']['FRD2']=0
model0['ub_list']['FRD3']=0
model0['lb_list']['FRD3']=0
model0['ub_list']['PTAr']=0
model0['lb_list']['PTAr']=0
model0['ub_list']['PTAr_reverse']=0
model0['lb_list']['PTAr_reverse']=0
model0['ub_list']['PTA2']=0
model0['lb_list']['PTA2']=0
model0['ub_list']['PFL']=0
model0['lb_list']['PFL']=0
model0['ub_list']['OBTFL']=0
model0['lb_list']['OBTFL']=0
model0['ub_list']['ACKr']=0
model0['lb_list']['ACKr']=0
model0['ub_list']['PTAr']=0
model0['lb_list']['PTAr']=0
model0['ub_list']['PTA2']=0
model0['lb_list']['PTA2']=0

In [6]:
model1['ub_list']['ACGAptspp']=0
model1['lb_list']['ACGAptspp']=0
model1['ub_list']['ACMANAptspp']=0
model1['lb_list']['ACMANAptspp']=0
model1['ub_list']['ACMUMptspp']=0
model1['lb_list']['ACMUMptspp']=0
model1['ub_list']['ARBTptspp']=0
model1['lb_list']['ARBTptspp']=0
model1['ub_list']['ASCBptspp']=0
model1['lb_list']['ASCBptspp']=0
model1['ub_list']['CHTBSptspp']=0
model1['lb_list']['CHTBSptspp']=0
model1['ub_list']['DHAPT']=0
model1['lb_list']['DHAPT']=0
model1['ub_list']['FRUpts2pp']=0
model1['lb_list']['FRUpts2pp']=0
model1['ub_list']['FRUptspp']=0
model1['lb_list']['FRUptspp']=0
model1['ub_list']['GALTptspp']=0
model1['lb_list']['GALTptspp']=0
model1['ub_list']['GAMptspp']=0
model1['lb_list']['GAMptspp']=0
model1['ub_list']['GLCptspp']=0
model1['lb_list']['GLCptspp']=0
model1['ub_list']['MALTptspp']=0
model1['lb_list']['MALTptspp']=0
model1['ub_list']['MANptspp']=0
model1['lb_list']['MANptspp']=0
model1['ub_list']['MNLptspp']=0
model1['lb_list']['MNLptspp']=0
model1['ub_list']['SBTptspp']=0
model1['lb_list']['SBTptspp']=0
model1['ub_list']['SUCptspp']=0
model1['lb_list']['SUCptspp']=0
model1['ub_list']['TREptspp']=0
model1['lb_list']['TREptspp']=0
model1['ub_list']['HEX1']=0
model1['lb_list']['HEX1']=0
model1['ub_list']['EX_glc__D_e_reverse']=0
model1['lb_list']['BIOMASS_Ec_iML1515_core_75p37M'] = 0.457*0.991 #0.19

### Predict the productions (g/L) under different inoculum ratios

In [7]:
#If you want to simulate the production with the unit of g/L, the real molar masses of exchange metabolites and carbon sources should be input, and the concentrations of exchange metabolites and carbon sources should be input with the unit of g/L
#If you want to simulate the production with the unit of mM, the molar masses of exchange metabolites and carbon sources should be input with the same value, and the concentrations of exchange metabolites and carbon sources should be input with the unit of mM

#model_list: the list of models calculated by the function of Get_Concretemodel_Need_Data
#target_list: the double list consist of the list of target reactions of each model, where the first reaction of each target reaction list is biomass biosynthesis reaction and the second is the reaction whose flux need to be maximized or minimized
#growth_list: the list of minimize growth rates in the function of MDF_Calculation
#growth_ratio: the list of ratios of lower bounds of growth rates to maximum growth rates
#parameter_list: the list of the total number of enzyme ratios, the upper bounds for substrate input reaction fluxes and the maximum values minus the minimum values of reaction thermodynamic driving forces
#substrate_environment: the list of the carbon source uptake reactions and the initial carbon source concentrations in environment
#metabolites_monitored: the list of metabolites delivered by the strains and the product
#metabolites_mass: the dictionary whose keys are the metabolites in metabolites_monitored and values are the molar masses of the metabolites
#concentrations_initial: the dictionary whose keys are the metabolites in metabolites_monitored and values are the initial concentrations of the metabolites in environment
#reactions_monitored: the list of dictionaries whose keys are the reactions of the metabolite uptake or release in metabolites_monitored and values are the ratios of lower bounds of the reaction fluxes to the max reaction fluxes
#substrate_mass: the list of the molar masses of the carbon sources
#t: the number of the time points for mono-cultivation
#culture_condition: the list of the inoculum ratio of the straints and time points for co-cultivation
#map_coefficient: the list of mapping constants of the instantaneous fluxes corresponding to the mean fluxes for the mono strains
#constraints: the constraints used in the simulation, where 'kinetics and thermodynamics' consist of FBA, kinetic and thermodynamic constraints, 'kinetics' consist of FBA and kinetic constraints, 'thermodynamics' consist of FBA and thermodynamic constraints and 'FBA' consist of FBA constraint

model_list=[model0, model1]
target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']]
growth_list=[0.01, 0.01]
growth_ratio=[0.965, 0.989]
parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]]
substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]]
metabolites_monitored=['ibutoh_e', 'isobt_e']
metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}
concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}
reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}]
substrate_mass=[180,150]
t=60
culture_condition=[[1/10, 9/10], 60]
map_coefficient = [2, 1.88]
constraints='FBA'

ppmcot(model_list, target_list, growth_list,growth_ratio, parameter_list, substrate_environment,  metabolites_monitored, metabolites_mass, concentrations_initial, reactions_monitored, substrate_mass, t, culture_condition, map_coefficient, constraints='FBA')

After 1 time point, the concentration of ibutoh_e is 0.007978929912804123 g/L or mM
After 1 time point, the concentration of isobt_e is 0.0004252397837788346 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.008927299591591527 g/L or mM
After 2 time point, the concentration of isobt_e is 0.0009099481310496644 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.00984606306574627 g/L or mM
After 3 time point, the concentration of isobt_e is 0.0014522685522002678 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.010736144581763987 g/L or mM
After 4 time point, the concentration of isobt_e is 0.0020504025135094067 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.011598439533015516 g/L or mM
After 5 time point, the concentration of isobt_e is 0.0027026076278770777 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.012433815360479371 g/L or mM
After 6 time point, the concentration of isobt_e is 0.0034071959020477867 g/L

After 50 time point, the concentration of ibutoh_e is 0.031936174167739875 g/L or mM
After 50 time point, the concentration of isobt_e is 0.06798477691914645 g/L or mM
After 51 time point, the concentration of ibutoh_e is 0.03213664659537688 g/L or mM
After 51 time point, the concentration of isobt_e is 0.06992485289027751 g/L or mM
After 52 time point, the concentration of ibutoh_e is 0.03233086067476663 g/L or mM
After 52 time point, the concentration of isobt_e is 0.07187710726880864 g/L or mM
After 53 time point, the concentration of ibutoh_e is 0.03251901177902686 g/L or mM
After 53 time point, the concentration of isobt_e is 0.07384115986921343 g/L or mM
After 54 time point, the concentration of ibutoh_e is 0.03270128918211475 g/L or mM
After 54 time point, the concentration of isobt_e is 0.07581664237460223 g/L or mM
After 55 time point, the concentration of ibutoh_e is 0.03287787624923167 g/L or mM
After 55 time point, the concentration of isobt_e is 0.0778031979662048 g/L or m

{'ibutoh_e': 0.0336814834791871, 'isobt_e': 0.08789034412976633}

In [7]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']], growth_list=[0.01, 0.01],growth_ratio=[0.965, 0.989], parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]], substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]],  metabolites_monitored=['ibutoh_e', 'isobt_e'], metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}, concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}, reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}], substrate_mass=[180,150], t=60, culture_condition=[[1/5, 4/5], 60], map_coefficient = [2, 1.88], constraints='FBA')

After 1 time point, the concentration of ibutoh_e is 0.009190189541201868 g/L or mM
After 1 time point, the concentration of isobt_e is 0.00042793686485919177 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.011311572087777829 g/L or mM
After 2 time point, the concentration of isobt_e is 0.0009897684219633929 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.013366309280269406 g/L or mM
After 3 time point, the concentration of isobt_e is 0.001681288235664228 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.015356494849122227 g/L or mM
After 4 time point, the concentration of isobt_e is 0.002498422019689134 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.017284156748143983 g/L or mM
After 5 time point, the concentration of isobt_e is 0.003437223485547467 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.019151259220945852 g/L or mM
After 6 time point, the concentration of isobt_e is 0.0044938703213471915 g/L 

After 50 time point, the concentration of ibutoh_e is 0.07044398719603209 g/L or mM
After 50 time point, the concentration of isobt_e is 0.11103779649194376 g/L or mM
After 51 time point, the concentration of ibutoh_e is 0.07144863575262969 g/L or mM
After 51 time point, the concentration of isobt_e is 0.11377273202954642 g/L or mM
After 52 time point, the concentration of ibutoh_e is 0.07245328430922443 g/L or mM
After 52 time point, the concentration of isobt_e is 0.11650766756715462 g/L or mM
After 53 time point, the concentration of ibutoh_e is 0.07345793286580636 g/L or mM
After 53 time point, the concentration of isobt_e is 0.11924260310478776 g/L or mM
After 54 time point, the concentration of ibutoh_e is 0.07446258142240396 g/L or mM
After 54 time point, the concentration of isobt_e is 0.12197753864239043 g/L or mM
After 55 time point, the concentration of ibutoh_e is 0.07546722997898873 g/L or mM
After 55 time point, the concentration of isobt_e is 0.12471247418001802 g/L or m

{'ibutoh_e': 0.08049047276197813, 'isobt_e': 0.13838715186802858}

In [8]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']], growth_list=[0.01, 0.01],growth_ratio=[0.965, 0.989], parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]], substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]],  metabolites_monitored=['ibutoh_e', 'isobt_e'], metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}, concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}, reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}], substrate_mass=[180,150], t=60, culture_condition=[[1/3, 2/3], 60], map_coefficient = [2, 1.88], constraints='FBA')

After 1 time point, the concentration of ibutoh_e is 0.010828830825547399 g/L or mM
After 1 time point, the concentration of isobt_e is 0.00043153297296725206 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.014536364290540536 g/L or mM
After 2 time point, the concentration of isobt_e is 0.0010991040529590096 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.01812644309536358 g/L or mM
After 3 time point, the concentration of isobt_e is 0.001995235552741214 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.021602788203649276 g/L or mM
After 4 time point, the concentration of isobt_e is 0.0031126866782177232 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.024969002698905018 g/L or mM
After 5 time point, the concentration of isobt_e is 0.004444446023644413 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.028228575518950912 g/L or mM
After 6 time point, the concentration of isobt_e is 0.005983724304614588 g/L o

After 51 time point, the concentration of ibutoh_e is 0.1579212867499642 g/L or mM
After 51 time point, the concentration of isobt_e is 0.10830910447031956 g/L or mM
After 52 time point, the concentration of ibutoh_e is 0.1607908246472734 g/L or mM
After 52 time point, the concentration of isobt_e is 0.11060736962796652 g/L or mM
After 53 time point, the concentration of ibutoh_e is 0.16366036254459457 g/L or mM
After 53 time point, the concentration of isobt_e is 0.11290563478559021 g/L or mM
After 54 time point, the concentration of ibutoh_e is 0.166529900441893 g/L or mM
After 54 time point, the concentration of isobt_e is 0.11520389994325812 g/L or mM
After 55 time point, the concentration of ibutoh_e is 0.16939943694114218 g/L or mM
After 55 time point, the concentration of isobt_e is 0.11750216782145433 g/L or mM
After 56 time point, the concentration of ibutoh_e is 0.1722689748384406 g/L or mM
After 56 time point, the concentration of isobt_e is 0.11980043297912224 g/L or mM
Aft

{'ibutoh_e': 0.18374712223349857, 'isobt_e': 0.12899350177135546}

In [7]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']], growth_list=[0.01, 0.01],growth_ratio=[0.965, 0.989], parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]], substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]],  metabolites_monitored=['ibutoh_e', 'isobt_e'], metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}, concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}, reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}], substrate_mass=[180,150], t=60, culture_condition=[[1/2, 1/2], 60], map_coefficient = [2, 1.88], constraints='FBA')

After 1 time point, the concentration of ibutoh_e is 0.012915106719967883 g/L or mM
After 1 time point, the concentration of isobt_e is 0.0004360281081040921 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.018640870873829644 g/L or mM
After 2 time point, the concentration of isobt_e is 0.0012405066151173923 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.024183353317125726 g/L or mM
After 3 time point, the concentration of isobt_e is 0.002401641423772016 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.029548420897412445 g/L or mM
After 4 time point, the concentration of isobt_e is 0.003908015965850263 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.03479258094519092 g/L or mM
After 5 time point, the concentration of isobt_e is 0.0056496700306472485 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.04003674053891883 g/L or mM
After 6 time point, the concentration of isobt_e is 0.007391324979002111 g/L or 

After 51 time point, the concentration of ibutoh_e is 0.27602393300284644 g/L or mM
After 51 time point, the concentration of isobt_e is 0.0857657767435007 g/L or mM
After 52 time point, the concentration of ibutoh_e is 0.28126809305062583 g/L or mM
After 52 time point, the concentration of isobt_e is 0.08750743080829591 g/L or mM
After 53 time point, the concentration of ibutoh_e is 0.2865122530984116 g/L or mM
After 53 time point, the concentration of isobt_e is 0.08924908487307878 g/L or mM
After 54 time point, the concentration of ibutoh_e is 0.2917564131461937 g/L or mM
After 54 time point, the concentration of isobt_e is 0.09099073893786871 g/L or mM
After 55 time point, the concentration of ibutoh_e is 0.2970005731939795 g/L or mM
After 55 time point, the concentration of isobt_e is 0.09273239300265158 g/L or mM
After 56 time point, the concentration of ibutoh_e is 0.30224473324176526 g/L or mM
After 56 time point, the concentration of isobt_e is 0.09447404706743445 g/L or mM
Af

{'ibutoh_e': 0.32322137343289287, 'isobt_e': 0.10144066332659593}

In [8]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_ibutoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_isobt_e']], growth_list=[0.01, 0.01],growth_ratio=[0.965, 0.989], parameter_list=[[0.19, 25, 1249], [0.19, 10, 1249]], substrate_environment=[['EX_glc__D_e_reverse',25],['EX_xyl__D_e_reverse',10]],  metabolites_monitored=['ibutoh_e', 'isobt_e'], metabolites_mass={'ibutoh_e': 74, 'isobt_e': 144}, concentrations_initial={'ibutoh_e': 0.007, 'isobt_e': 0}, reactions_monitored=[{'EX_ibutoh_e': 0}, {'EX_ibutoh_e_reverse': 0, 'EX_isobt_e': 0}], substrate_mass=[180,150], t=60, culture_condition=[[2/3, 1/3], 60], map_coefficient = [2, 1.88], constraints='FBA')

After 1 time point, the concentration of ibutoh_e is 0.015043576268821102 g/L or mM
After 1 time point, the concentration of isobt_e is 0.00044052324324096834 g/L or mM
After 2 time point, the concentration of ibutoh_e is 0.022827023281108427 g/L or mM
After 2 time point, the concentration of isobt_e is 0.001387243958655769 g/L or mM
After 3 time point, the concentration of ibutoh_e is 0.030494150145320614 g/L or mM
After 3 time point, the concentration of isobt_e is 0.0025603173946492216 g/L or mM
After 4 time point, the concentration of ibutoh_e is 0.03816127731535564 g/L or mM
After 4 time point, the concentration of isobt_e is 0.0037333902355279613 g/L or mM
After 5 time point, the concentration of ibutoh_e is 0.04582840448538944 g/L or mM
After 5 time point, the concentration of isobt_e is 0.004906463076409078 g/L or mM
After 6 time point, the concentration of ibutoh_e is 0.05349553165542263 g/L or mM
After 6 time point, the concentration of isobt_e is 0.006079535917291382 g/L or 

After 51 time point, the concentration of ibutoh_e is 0.39851624553996084 g/L or mM
After 51 time point, the concentration of isobt_e is 0.05886783081701715 g/L or mM
After 52 time point, the concentration of ibutoh_e is 0.40618337240417424 g/L or mM
After 52 time point, the concentration of isobt_e is 0.060040904253008225 g/L or mM
After 53 time point, the concentration of ibutoh_e is 0.41385049926838763 g/L or mM
After 53 time point, the concentration of isobt_e is 0.0612139776889993 g/L or mM
After 54 time point, the concentration of ibutoh_e is 0.421517626132601 g/L or mM
After 54 time point, the concentration of isobt_e is 0.06238705112499037 g/L or mM
After 55 time point, the concentration of ibutoh_e is 0.4291847529968144 g/L or mM
After 55 time point, the concentration of isobt_e is 0.06356012456098145 g/L or mM
After 56 time point, the concentration of ibutoh_e is 0.4368518801668482 g/L or mM
After 56 time point, the concentration of isobt_e is 0.06473319740186256 g/L or mM
Af

{'ibutoh_e': 0.46752038854116673, 'isobt_e': 0.06942548936048984}